In [1]:
import numpy as np
import sys

In [2]:
verbosity = False

def info(text):
    if verbosity:
        print(text)

In [18]:
class Generator:
    def __init__(self,intensity):
        self.intensity = intensity
        
    def get_time(self):
        return np.random.exponential(1 / self.intensity)
    
    def get_events(self, count):
        timings = []
        timings.append(Event('GENERATOR', self.get_time())) 
        for i in range(1, count):
            timings.append(Event('GENERATOR', timings[-1].time_value + self.get_time())) 
        return timings   
        
class Queue:
    def __init__(self, pick_fastest=False):
        self.queue = []
        self.len_times = []
        self.time_of_change = 0
        self.pick_fastest = pick_fastest
        
    @property
    def count(self):
        return len(self.queue)
    
    def update_times(self,time):
        self.len_times.append((self.count, time - self.time_of_change))        
        self.time_of_change = time        
    
    
    def add(self, order, time):        
        self.update_times(time)                
        self.queue.append(order)
        info(f"QUEUE - added order at {time}; ({self.count})")

        
    def pop(self, time):        
        info(f"QUEUE - poped order at {time}; ({self.count-1})")             
        self.update_times(time)   
        idx = 0
        if self.pick_fastest:
            idx = self.queue.index(min(self.queue, key=lambda x: x.time_needed))                
        return self.queue.pop(idx)
    
    def inc(self, time):
        for order in self.queue:
            order.inc_queue(time)
            
    def get_avg_len(self, job_time):
        result = 0
        for length, time in self.len_times:
            result += length * time
        info("--List of queue times--")
        info(self.len_times[:100])
        in
        return result / job_time
            
class Processor:
    def __init__(self, intensity):
        self.intensity = intensity
        self.current_order = None
        self.current_time = 0
        self.process_time = sys.float_info.max
        
    def start(self, order, time=None):
        self.current_order = order
        interval = np.random.exponential(1 / self.intensity) if order.time_needed == -1 else order.time_needed
        self.current_time = self.current_time if time is None else time
        self.process_time = self.current_time + interval
        self.current_order.inc_system(interval)
        info(f"PROCESSOR - started work at {self.current_time}")
        return Event('PROCESSOR', self.process_time)
    
    def end(self):
        self.current_time = self.process_time
        order = self.current_order
        self.current_order = None
        return order
        
    
    def check_end(self, time):
        if time > self.process_time:
            self.current_time = self.process_time
            order = self.current_order
            self.current_order = None
            info(f"PROCESSOR - ended work at {self.current_time}")
            return order
        return None
            
    def is_free(self):
        return self.current_order is None    
    

class Event:
    def __init__(self, event_type, time_value):
        self.type = event_type
        self.time_value = time_value
    def __lt__(self, other):
        return self.time_value < other.time_value

class Order:
    def __init__(self, time, generate_time = False):
        self.entry_time = time 
        self.time_in_queue = 0
        self.time_in_system = 0
        self.time_needed = -1
        if generate_time:
            self.time_needed = np.random.exponential(1 / 3)
        
    def inc_queue(self, time):
        self.time_in_queue += time
        self.time_in_system = self.time_in_queue
        
    def inc_system(self, time):
        self.time_in_system += time
                    

In [21]:
import bisect

count = 100000
input_intensity = 2.5
output_intensity = 3

    
class System:
    def __init__(self, input_intensity=input_intensity,output_intensity=output_intensity, orders_with_time=False,seed=None):
        self.time = 0
        self.orders_with_time = orders_with_time
        self.output_orders = []
        
        self.generator = Generator(input_intensity)
        self.processor = Processor(output_intensity)
        self.queue = Queue(self.orders_with_time)
        
        np.random.seed(seed)

    def on(self, count=count):
        
        events = self.generator.get_events(count)
        self.time = events[0].time_value
        
        final_value = events[-1].time_value
        process_event = self.processor.start(Order(self.time, self.orders_with_time), self.time)
        bisect.insort_left(events, process_event)
        i = 1
      
        while self.time < final_value:                   
            self.time = events[i].time_value
            info(f"SYSTEM - current time: {self.time} -- {events[i].type}")
            self.queue.inc(self.time - events[i-1].time_value)

            if events[i].type == 'GENERATOR':
                new_order = Order(self.time, self.orders_with_time)
                if self.processor.is_free():
                    process_event = self.processor.start(new_order, self.time)
                    bisect.insort_left(events, process_event)
                else:
                    self.queue.add(new_order, self.time)
            if events[i].type == 'PROCESSOR':
                processed_order = self.processor.end()    
                self.output_orders.append(processed_order)
                
                if self.queue.count > 0:
                    process_event = self.processor.start(self.queue.pop(self.time),self.time)
                    bisect.insort_left(events, process_event)
            i += 1
                
        print(f"L={self.queue.get_avg_len(self.time)}")
        print(f"Wq={self.get_avg_order_time_in_queue()}")
        print(f"Ws={self.get_avg_order_time_in_system()}")
        print(f"Job time: {self.time}")  
        print(f"Output order amount: {len(self.output_orders)}")


    def get_avg_order_time_in_queue(self):
        result = 0
        for order in self.output_orders:
            result += order.time_in_queue
        return result / len(self.output_orders)
    
    def get_avg_order_time_in_system(self):
        result = 0
        for order in self.output_orders:
            result += order.time_in_system
        return result / len(self.output_orders)



In [27]:
print("---FIFO---")
s1 = System(orders_with_time=False)
s1.on()
print("---Time dependent---")
s2 = System(orders_with_time=True)
s2.on()

---FIFO---
L=4.182472486616473
Wq=1.6721517868089777
Ws=2.005956813325098
Job time: 39978.90928199255
Output order amount: 99994
---Time dependent---
L=1.7904986043805222
Wq=0.7171029803304049
Ws=1.0503299117987115
Job time: 40049.543676662004
Output order amount: 99997
